In [ ]:
%%capture
pip install numpy pandas

In [ ]:
import numpy as np
import pandas as pd

### **Cohorts and academies_alias**

In [ ]:
cohorts = pd.read_csv('../sources/cohorts.csv')

In [ ]:
cohorts

In [ ]:
cohorts.info()

In [ ]:
academies_alias = pd.read_csv('../sources/academies_alias.csv')

In [ ]:
academies_alias.info()

In [ ]:
academies_alias

In [ ]:
academies_alias = academies_alias.rename(columns={'name':'campus'}).drop(columns=['slug', 'status']).drop_duplicates()

In [ ]:
academies_alias

In [ ]:
cohorts = cohorts.merge(academies_alias, how='left', on='academy_id')

In [ ]:
cohorts

In [ ]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [ ]:
cohorts['kickoff_date'] = cohorts['kickoff_date'].apply(correct_date)
cohorts['ending_date'] = cohorts['ending_date'].apply(correct_date)
cohorts['created_at'] = cohorts['created_at'].apply(correct_date)
cohorts['updated_at'] = cohorts['updated_at'].apply(correct_date)

In [ ]:
cohorts['datetime_kickoff'] = pd.to_datetime(cohorts['kickoff_date'])

In [ ]:
current_time = pd.Timestamp.now()

In [ ]:
upcoming_cohorts = cohorts[cohorts['datetime_kickoff']>current_time]

In [ ]:
upcoming_cohorts = upcoming_cohorts[upcoming_cohorts['never_ends']==False]

In [ ]:
upcoming_cohorts.info()

In [ ]:
upcoming_cohorts['stage'].value_counts()

In [ ]:
upcoming_cohorts = upcoming_cohorts[(upcoming_cohorts['stage']!='DELETED')]

In [ ]:
upcoming_cohorts.columns

In [ ]:
upcoming_cohorts

### **Unconfirmed**

In [ ]:
form_entry = pd.read_csv('../output/form_entry_processed.csv', low_memory=False)

In [ ]:
form_entry.columns

In [ ]:
form_entry['datetime_update'] = pd.to_datetime(form_entry['updated_at'])

In [ ]:
dict_assigned = form_entry.groupby('ac_expected_cohort')['email'].apply(list).to_dict()

### **Confirmed**

In [ ]:
cohort_users = pd.read_csv('../sources/cohort_users.csv')

In [ ]:
cohort_users.columns

In [ ]:
cohorts_slugs = cohorts[['id', 'slug']].drop_duplicates()

In [ ]:
cohort_users = cohort_users.merge(cohorts_slugs, left_on='cohort_id', right_on='id', how='left')
cohort_users = cohort_users.rename(columns={'id_x':'id'})
cohort_users = cohort_users.drop(columns=['id_y'])

In [ ]:
auth = pd.read_csv('../sources/auth_user.csv')

In [ ]:
cohort_users_auth = cohort_users.merge(auth, left_on='user_id', right_on='id', how='left')

In [ ]:
cohort_users_auth.sample(3)

In [ ]:
cohort_users_auth.columns

In [ ]:
cohort_users_auth['role'].value_counts()

In [ ]:
cohort_users_auth = cohort_users_auth[cohort_users_auth['role'].str.lower()=='student']

In [ ]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [ ]:
cohort_users_auth['updated_at'] = cohort_users_auth['updated_at'].apply(correct_date)

In [ ]:
cohort_users_auth['datetime_update'] = pd.to_datetime(cohort_users_auth['updated_at'])

In [ ]:
dict_confirmed = cohort_users_auth.groupby('slug')['email'].apply(list).to_dict()

**Create final dataframe**

In [ ]:
list_slugs = list(dict_assigned.keys()) + list(dict_confirmed.keys())

In [ ]:
list_slugs = list(set(list_slugs))

In [ ]:
list_slugs

In [ ]:
dict_slug_email = {}

for slug in list_slugs:
    
    dict_slug_email[slug] = []

    if slug in list(dict_assigned.keys()):
        dict_slug_email[slug].extend(dict_assigned[slug])
    
    if slug in list(dict_confirmed.keys()):
        dict_slug_email[slug].extend(dict_confirmed[slug])

In [ ]:
dict_slug_email

In [ ]:
list_slug_email = []
for key, value in dict_slug_email.items():
    for item in value:
        list_slug_email.append((key, item))

In [ ]:
df_slug_email = pd.DataFrame(list_slug_email, columns=['slug', 'email'])

In [ ]:
df_slug_email

In [ ]:
upcoming_cohorts_users = upcoming_cohorts.merge(df_slug_email, on='slug', how='left')

In [ ]:
upcoming_cohorts_users

Add columns related to assigned

In [ ]:
def check_in_dict(row, dict_x):
    if row['slug'] not in dict_x:
        return False
    else:
        return row['email'] in dict_x[row['slug']]

In [ ]:
upcoming_cohorts_users['assigned'] = upcoming_cohorts_users.apply(lambda row: check_in_dict(row, dict_assigned), axis=1)

In [ ]:
upcoming_cohorts_users

In [ ]:
df_assigned = upcoming_cohorts_users[upcoming_cohorts_users['assigned']==True]

In [ ]:
df_assigned = pd.merge(df_assigned, form_entry, left_on=['slug', 'email'], right_on=['ac_expected_cohort', 'email'])

In [ ]:
df_assigned.columns

In [ ]:
grouped_assigned = df_assigned.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'location': 'last', 'ac_deal_id':'last', 'ac_deal_owner_full_name':'last', 'created_at_y': 'last'})
grouped_assigned = grouped_assigned.reset_index()

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users.merge(grouped_assigned, on='email', how='left').drop(columns=['slug_y', 'datetime_update'])

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug', 'created_at_y':'form_created_at', 'location':'expected_location', 'ac_deal_id':'deal_id', 'ac_deal_owner_full_name':'deal_owner'})

In [ ]:
upcoming_cohorts_users

Add columns related to confirmed

In [ ]:
upcoming_cohorts_users['confirmed'] = upcoming_cohorts_users.apply(lambda row: check_in_dict(row, dict_confirmed), axis=1)

In [ ]:
df_confirmed = upcoming_cohorts_users[upcoming_cohorts_users['confirmed']==True]

In [ ]:
# df_confirmed = pd.merge(upcoming_cohorts_users, cohort_users_auth, left_on=['slug', 'email'], right_on=['slug', 'email'])
df_confirmed = pd.merge(df_confirmed, cohort_users_auth, left_on=['slug', 'email'], right_on=['slug', 'email'])

In [ ]:
df_confirmed.columns

In [ ]:
grouped_confirmed = df_confirmed.groupby(['slug', 'email']).agg({'datetime_update': 'max', 'educational_status': 'last', 'created_at_y': 'last'})
grouped_confirmed = grouped_confirmed.reset_index()

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users.merge(grouped_confirmed, on='email', how='left')

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users.rename(columns={'slug_x':'slug', 'created_at_y':'inscription_created_at'})

In [ ]:
upcoming_cohorts_users

**Cleaning upcoming_cohorts**

In [ ]:
upcoming_cohorts_users['days_until_kickoff'] = (upcoming_cohorts_users['datetime_kickoff'] - pd.Timestamp.now()).dt.days

In [ ]:
upcoming_cohorts_users['form_created_at'] = upcoming_cohorts_users['form_created_at'].apply(correct_date)
upcoming_cohorts_users['inscription_created_at'] = upcoming_cohorts_users['inscription_created_at'].apply(correct_date)

In [ ]:
upcoming_cohorts_users['form_created_at'] = pd.to_datetime(upcoming_cohorts_users['form_created_at'])
upcoming_cohorts_users['inscription_created_at'] = pd.to_datetime(upcoming_cohorts_users['inscription_created_at'])

In [ ]:
upcoming_cohorts_users['days_since_form_creation'] = (pd.Timestamp.now()- upcoming_cohorts_users['form_created_at']).dt.days
upcoming_cohorts_users['days_since_inscription'] = (pd.Timestamp.now()- upcoming_cohorts_users['inscription_created_at']).dt.days

In [ ]:
def create_link(row):
    if pd.isnull(row['deal_id']):
        return np.nan
    else:
        return 'https://4geeks.activehosted.com/app/deals/' + str(int(row['deal_id']))

In [ ]:
upcoming_cohorts_users['ac_deal_link'] = upcoming_cohorts_users.apply(create_link, axis=1)

In [ ]:
upcoming_cohorts_users['assigned'] = np.where(upcoming_cohorts_users['assigned'] == True, 1, 0)

In [ ]:
upcoming_cohorts_users['confirmed'] = np.where(upcoming_cohorts_users['confirmed'] == True, 1, 0)

In [ ]:
upcoming_cohorts_users['unconfirmed'] = upcoming_cohorts_users['assigned']

In [ ]:
upcoming_cohorts_users.loc[upcoming_cohorts_users['confirmed'] == 1, 'unconfirmed'] = 0

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users[['id', 'slug', 'name', 'stage', 'language', 'academy_id', 'campus', 'datetime_kickoff', 'days_until_kickoff', 'email', 'assigned', 'expected_location', 'ac_deal_link', 'deal_owner', 'form_created_at', 'days_since_form_creation', 'unconfirmed', 'confirmed', 'educational_status', 'inscription_created_at', 'days_since_inscription']]

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users.drop_duplicates()

In [ ]:
upcoming_cohorts_users = upcoming_cohorts_users.sort_values(by=['days_until_kickoff'], ascending=True)

In [ ]:
upcoming_cohorts_users

In [ ]:
upcoming_cohorts_users.to_csv('../output/upcoming_cohorts_users.csv')